In [2]:
!ls eventTiming/

AUS AUT AZE BHR CAN CHN ESP GBR HUN MCO RUS


In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
url='http://www.fia.com/events/fia-formula-one-world-championship/season-2017/formula-one'
response = requests.get(url,params={})

In [5]:
soup=BeautifulSoup(response.content)
events=soup.findAll('div',{'class':'event-item past'})

In [6]:
len(events)

20

In [6]:
events

[<div class="event-item past">
 <div class="inner">
 <div class="date cell"> <div class="date-inner">
 <div class="from-date cell">
 <div class="day">26</div>
 <div class="month">Mar</div>
 </div>
 </div> </div>
 <div class="event cell"> <a href="/championship/events/fia-formula-one-world-championship/season-2017/australian-grand-prix"><div class="event-name cell">Australian Grand Prix</div>
 <div class="event-location">ALBERT PARK</div></a> </div>
 <div class="event-right-wrapper car-category-empty">
 <div>
 <div class="classification cell ">
 <div class="classification-link">
 <a class="link-label" href="/events/fia-formula-one-world-championship/season-2017/race-classification-19">Classification</a> </div>
 </div>
 <div class="car-category empty">
 </div>
 <div class="country cell">
 <img alt="AU" class="countryicon iconset-gosquared_flat_large countrycode-au" src="http://www.fia.com/sites/all/modules/contrib/countryicons_gosquared/flat-large/au.png" title=""/> <div class="country-n

In [5]:
!ls eventTiming/

ls: eventTiming/: No such file or directory


In [6]:
events

[<div class="event-item past">
 <div class="inner">
 <div class="date cell"> <div class="date-inner">
 <div class="from-date cell">
 <div class="day">20</div>
 <div class="month">Mar</div>
 </div>
 </div> </div>
 <div class="event cell"> <a href="/championship/events/fia-formula-one-world-championship/season-2016/grand-prix-australia"><div class="event-name cell">Grand Prix of Australia</div>
 <div class="event-location">ALBERT PARK</div></a> </div>
 <div class="classification cell"> <a class="link-label" href="/events/fia-formula-one-world-championship/season-2016/race-classification">Classification</a> </div>
 <div class="country cell"> <div><img alt="AUS" src="http://www.fia.com/sites/all/modules/contrib/countryicons_gosquared/flat-large/au.png" title="AUS"/> <div class="country-name">AUS</div>
 </div> </div> </div>
 </div>, <div class="event-item past">
 <div class="inner">
 <div class="date cell"> <div class="date-inner">
 <div class="from-date cell">
 <div class="day">03</div>
 <

In [10]:
timing=[x.lower() for x in ['Official Classification','Provisional Classification','Preliminary Classification',
        'Fastest Laps','History Chart','Lap Chart','Lap Analysis','Pit Stop Summary','Best Sector Times'
        ,'Maximum Speeds','Official Starting Grid','Provisional Starting Grid','Qualifying Session',
        'Official Classification','Provisional Classification','Speed Trap',
        'Preliminary Classification','Lap Times','Provisionnal Classification','Provisionnal Starting Grid',
        'Speed Trap','Classification']]


In [8]:
events[-6:]

[<div class="event-item past">
 <div class="inner">
 <div class="date cell"> <div class="date-inner">
 <div class="from-date cell">
 <div class="day">01</div>
 <div class="month">Oct</div>
 </div>
 </div> </div>
 <div class="event cell"> <a href="/championship/events/fia-formula-one-world-championship/season-2017/malaysian-grand-prix"><div class="event-name cell">Malaysian Grand Prix</div>
 <div class="event-location">SEPANG</div></a> </div>
 <div class="event-right-wrapper car-category-empty">
 <div>
 <div class="classification cell ">
 <div class="classification-link">
 <a class="link-label" href="/events/fia-formula-one-world-championship/season-2017/race-classification-34">Classification</a> </div>
 </div>
 <div class="car-category empty">
 </div>
 <div class="country cell">
 <img alt="MY" class="countryicon iconset-gosquared_flat_large countrycode-my" src="https://www.fia.com/sites/all/modules/contrib/countryicons_gosquared/flat-large/my.png" title=""/> <div class="country-name">M

In [11]:
import os
domain="http://www.fia.com"

import re

def download_file(url,f):
    response = re.urlopen(url)
    file = open("document.pdf", 'w')
    file.write(response.read())
    file.close()
    print("Completed")

for event in events[-6:]:
    prev=''
    phase='R'
    gp=event.find('div',{'class':'event-name'}).text.replace('Grand Prix of ','').strip()
    c=event.find('div',{'class':'country-name'}).text.strip()
    l=event.find('div',{'class':'event-location'}).text.strip()
    u=event.find('div',{'class':'event cell'}).find('a')['href']
    
    print(gp,c,l)
    d='eventTiming/{}'.format(c)
    if not os.path.exists(d):
        os.makedirs(d) 
    url='{}{}'.format(domain,u)
    print(url)
    eventpage=requests.get(url)
    eventsoup=BeautifulSoup(eventpage.content)
    #print(eventsoup.find('a',text='Event & Timing information')['href'])
    #pdfsoup=BeautifulSoup(requests.get('{}{}'.format(domain,eventsoup.find('a',text='Event & Timing information')['href'])).content)
    #soup.findAll('a', href=re.compile('^http://www.nhl.com/ice/boxscore.htm\?id='))
    eventSlug='.*eventtiming-.*'
    print(eventsoup.find('a',href=re.compile(eventSlug))['href'])
    pdfsoup=BeautifulSoup(requests.get('{}{}'.format(domain,eventsoup.find('a',href=re.compile(eventSlug))['href'])).content)
    middle=pdfsoup.find('div',{'class':'content'}).find('div',{'class':'middle'})

    for maybe in middle.findAll("a"):
        #print(maybe['href'])
        prepath=maybe.text.encode('ascii', 'ignore').strip()
        #print('fing path*{}*'.format(prepath))
        if prepath.lower() in timing:
            if (prev=='Official Starting Grid' and prepath=='Lap Times'): phase='P3'
            path='{} - {}'.format(phase,prepath)
            if prepath=='Provisional Starting Grid' or prepath=='Provisionnal Starting Grid':
                phase='Q'
            elif (phase=='Q' and prepath=='Maximum Speeds') or (phase=='Q' and (c=='MYS' or c=='CHN') and prepath=='Provisionnal Classification'):
                phase='P3'
            elif phase=='P3' and prepath=='Classification' :
                phase='P2'
            elif phase=='P2' and prepath=='Classification':
                phase='P1'
        else:
            path=prepath
        prev=prepath
        p='{d}/{path}.pdf'.format(d=d,path=path.replace('/','_'))

        #print(p)
        if not os.path.isfile(p):
            print("downloading",d,maybe.text.encode('ascii', 'ignore'))
            os.system('wget {url} -O "{p}"'.format(url='{}{}'.format(domain,maybe['href']),p=p))


(u'Malaysian Grand Prix', u'MYS', u'SEPANG')
http://www.fia.com/championship/events/fia-formula-one-world-championship/season-2017/malaysian-grand-prix
/events/fia-formula-one-world-championship/season-2017/eventtiming-information-2
('downloading', 'eventTiming/MYS', 'Start time information')
('downloading', 'eventTiming/MYS', 'Grand Prix Preview')
('downloading', 'eventTiming/MYS', 'Entry List')
('downloading', 'eventTiming/MYS', 'Circuit Map')
('downloading', 'eventTiming/MYS', 'Official Classification')
('downloading', 'eventTiming/MYS', 'Provisional Classification')
('downloading', 'eventTiming/MYS', 'Fastest Laps')
('downloading', 'eventTiming/MYS', 'Speed Trap')
('downloading', 'eventTiming/MYS', 'History Chart')
('downloading', 'eventTiming/MYS', 'Lap Chart')
('downloading', 'eventTiming/MYS', 'Lap Analysis')
('downloading', 'eventTiming/MYS', 'Pit Stop Summary')
('downloading', 'eventTiming/MYS', 'Best Sector Times')
('downloading', 'eventTiming/MYS', 'Maximum Speeds')
('downlo

('downloading', 'eventTiming/JPN', 'Post Race Press Conference')
('downloading', 'eventTiming/JPN', 'Media Kit')
(u'United States Grand Prix', u'USA', u'AUSTIN')
http://www.fia.com/championship/events/fia-formula-one-world-championship/season-2017/united-states-grand-prix
/events/fia-formula-one-world-championship/season-2017/eventtiming-information-35
('downloading', 'eventTiming/USA', 'Start time information')
('downloading', 'eventTiming/USA', 'Grand Prix Preview')
('downloading', 'eventTiming/USA', 'Entry List')
('downloading', 'eventTiming/USA', "Race Director's Note Doc8 - Revised Circuit Map")
('downloading', 'eventTiming/USA', 'Circuit Map')
('downloading', 'eventTiming/USA', 'Official Classification')
('downloading', 'eventTiming/USA', 'Provisional Classification')
('downloading', 'eventTiming/USA', 'Fastest Laps')
('downloading', 'eventTiming/USA', 'Speed Trap')
('downloading', 'eventTiming/USA', 'History Chart')
('downloading', 'eventTiming/USA', 'Lap Chart')
('downloading',

('downloading', 'eventTiming/MEX', 'Stewards Decision Doc17 - Haas F1 Team')
('downloading', 'eventTiming/MEX', 'Stewards Decision Doc14 - K.Magnussen')
('downloading', 'eventTiming/MEX', 'Stewards Decision Doc12 - P.Gasly')
('downloading', 'eventTiming/MEX', 'Stewards Decision Doc11 - S.Vandoorne')
('downloading', 'eventTiming/MEX', 'Stewards Decision Doc9 - F.Alonso')
('downloading', 'eventTiming/MEX', 'Summons Doc29 - V.Bottas')
('downloading', 'eventTiming/MEX', 'Summons Doc28 - M.Verstappen ')
('downloading', 'eventTiming/MEX', 'Note to Teams')
('downloading', 'eventTiming/MEX', 'Stewards Information Doc3 - Scrutineers')
('downloading', 'eventTiming/MEX', 'Event Notes')
('downloading', 'eventTiming/MEX', 'Press Conference Schedule')
('downloading', 'eventTiming/MEX', 'Thursday Press Conference')
('downloading', 'eventTiming/MEX', 'Friday Press Conference ')
('downloading', 'eventTiming/MEX', 'Post Qualifying Press Conference')
('downloading', 'eventTiming/MEX', 'Post Race Press Co

('downloading', 'eventTiming/ARE', 'Technical Report Doc5 - Initial Scrutineering')
('downloading', 'eventTiming/ARE', 'Technical Report Doc4 - PU Elements')
('downloading', 'eventTiming/ARE', 'T.Mayer, E.Spano, E.Pirro')
('downloading', 'eventTiming/ARE', 'Stewards Decision Doc36 -C.Sainz')
('downloading', 'eventTiming/ARE', 'Stewards Decision Doc32 - N.Hulkenberg')
('downloading', 'eventTiming/ARE', 'Stewards Decision Doc23 - S.Vandoorne')
('downloading', 'eventTiming/ARE', 'Stewards Decision Doc12 - B.Hartley')
('downloading', 'eventTiming/ARE', 'Stewards Decision Doc11 - B.Hartley')
('downloading', 'eventTiming/ARE', 'Note to Teams')
('downloading', 'eventTiming/ARE', 'Event Notes (Revised)')
('downloading', 'eventTiming/ARE', 'Event Notes')
('downloading', 'eventTiming/ARE', 'Press Conference Schedule')
('downloading', 'eventTiming/ARE', 'Thursday Press Conference')
('downloading', 'eventTiming/ARE', 'Friday Press Conference')
('downloading', 'eventTiming/ARE', 'Post Qualifying Pr

In [44]:
ls

01_australia_eng.pdf
02_malesia_eng.pdf
PDF scraper.ipynb
Pirelli2015MediaGuide.pdf
eventTiming/
pirelli_preview_australia_25311.pdf
pirelli_preview_china_25671.pdf
pirelli_preview_malaysia_25495.pdf
pirelli_review_australia_25422.pdf
pirelli_review_malaysia_25605.pdf
rsf1-presskit-2015-en_v6.pdf
ubs-formula-1-f1-race-strategy-briefing-infographic-kuala lumpur-malaysia.pdf


In [12]:
print(url)
eventpage=requests.get(url)
eventsoup=BeautifulSoup(eventpage.content)
pdfsoup=BeautifulSoup(requests.get('{}{}'.format(domain,eventsoup.find('a',href=re.compile('.*event-timing-.*'))['href'])).content)
middle=pdfsoup.find('div',{'class':'content'}).find('div',{'class':'middle'})

http://www.fia.com/championship/events/fia-formula-one-world-championship/season-2016/grand-prix-russia


In [14]:
for a in middle.findAll("a"):
    print(a)

<a href="/file/40855/download?token=o-TNemmy" target="_blank">Grand Prix Preview</a>
<a href="/file/40904/download?token=MyE12zD6" target="_blank">Entry List</a>
<a href="/file/40881/download?token=4Rd5HCsi" target="_blank">Circuit Map</a>
<a href="/file/41063/download?token=B4SJLYk_" target="_blank">Official Classification</a>
<a href="/file/41062/download?token=db9G0SNt" target="_blank">Provisional Classification</a>
<a href="/file/41061/download?token=Wtwsqzec" target="_blank">Preliminary Classification</a>
<a href="/file/41060/download?token=Ukepyc88" target="_blank">Fastest Laps</a>
<a href="/file/41059/download?token=56tZIA-X" target="_blank">History Chart</a>
<a href="/file/41058/download?token=y8hjQG3p" target="_blank">Lap chart</a>
<a href="/file/41057/download?token=2CoHjHbX" target="_blank">Lap Analysis</a>
<a href="/file/41056/download?token=yS22U3BJ" target="_blank">Pit Stop summary</a>
<a href="/file/41055/download?token=rQbhefj7" target="_blank">Best Sector Times</a>
<a 